In [ ]:
import numpy as np, pandas as pd, requests

In [ ]:
monthlyInflation=pd.read_csv('inflationCPI_OECD_monthly.csv')
monthlyInflation
usefulMonthlyInflation=monthlyInflation[['LOCATION','TIME','Value']]
# As in the regression Notebook, we are selecting the useful columns of data: the country, date and inflation rate.

In [ ]:
usefulMonthlyInflation['TIME'] = pd.to_datetime(usefulMonthlyInflation['TIME'])
# This converts the dates we have into actual datetime format.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Analysed_usefulMonthlyInflation = usefulMonthlyInflation[usefulMonthlyInflation['LOCATION'].str.contains('TUR|DEU|USA|JPN')]
Analysed_usefulMonthlyInflation.reset_index()
# This creates a new dataframe only including the 4 countries that we are analysing (by identifying strings that contain their country code).
# The '|' operator represents 'or' in this context, hence this is used within contains().

,index,LOCATION,TIME,Value
0,0,DEU,2015-01-01,-0.283286
1,1,DEU,2015-02-01,-0.046563
2,2,DEU,2015-03-01,0.174787
3,3,DEU,2015-04-01,0.866233
4,4,DEU,2015-05-01,1.162551
...,...,...,...,...
326,409,USA,2021-07-01,5.365475
327,410,USA,2021-08-01,5.251272
328,411,USA,2021-09-01,5.390349
329,412,USA,2021-10-01,6.221869


In [ ]:
pivoted_monthlyInflation = pd.pivot_table(Analysed_usefulMonthlyInflation, values='Value', index='TIME', columns='LOCATION', aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False).reset_index()[22:89].reset_index()
pivoted_monthlyInflation
# From the previous dataframe, we create a Pandas pivot table using the date as the index and pivoting the different countries in 'LOCATION' as columns.
# This results in the date, as well as each country, being a column in the dataframe.
# We also only take observations from the 22nd row onwards as this is when the Ethereum price data begins (i.e., 31/10/2016).

LOCATION,index,TIME,DEU,JPN,TUR,USA
0,22,2016-11-01,0.802407,0.5,7.000521,1.692537
1,23,2016-12-01,1.504514,0.3,8.533056,2.074622
2,24,2017-01-01,1.616162,0.4,9.218773,2.500042
3,25,2017-02-01,1.913394,0.3,10.128290,2.737958
4,26,2017-03-01,1.400000,0.2,11.291790,2.380612
...,...,...,...,...,...,...
56,78,2021-07-01,3.770028,-0.3,18.951680,5.365475
57,79,2021-08-01,3.867924,-0.4,19.252660,5.251272
58,80,2021-09-01,4.064272,0.2,19.582570,5.390349
59,81,2021-10-01,4.532578,0.1,19.890020,6.221869


In [ ]:
url ='https://raw.githubusercontent.com/tm19784/tm19784.github.io/main/jsonFolder/rawPriceBTC.json'
html = requests.get(url)
Bitcoin_price_json = html.json()
BTC=pd.DataFrame(Bitcoin_price_json)
BTC_useful=BTC[['time_close','price_close']]
BTC_useful
# As previously highlighted, this raw file from my GitHub is copied from the CoinAPI, when I set it to return data for Bitcoin's price in pounds over 10-day intervals.
# I turn this JavaScript into a Pandas dataframe, and then select the time and price columns as the useful parts of the dataframe.

,time_close,price_close
0,2015-04-29T16:21:00.0000000Z,146.12
1,2015-05-09T20:13:18.0000000Z,154.44
2,2015-05-19T22:52:30.5100000Z,149.86
3,2015-05-29T19:45:01.0000000Z,154.88
4,2015-06-08T22:20:07.0000000Z,149.16
...,...,...
238,2021-11-03T23:59:59.1590000Z,45971.73
239,2021-11-13T23:59:53.8400240Z,48097.75
240,2021-11-23T23:59:57.5870000Z,43078.60
241,2021-12-03T23:59:57.3880000Z,40643.28


In [ ]:
BTC_useful['time_close'] = pd.to_datetime(BTC_useful['time_close'])
BTC_useful
BTC_monthly = BTC_useful.set_index('time_close').resample('1M').mean().reset_index()[18:79].reset_index()
BTC_monthly
# As highlighted in other notebooks, this first converts the time column in the dataframe into datetime format.
# I then resample the data to be monthly (with the mean in place), as this is not an option for the CoinAPI and this is the frequency of the unemployment data.
# The resample function requires a time index, thus 'time_close' is set as the index beforehand.
# Rows 18-79 are included as these are the rows for which each of our six variables do not have missing data.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,time_close,price_close
0,18,2016-10-31 00:00:00+00:00,532.686667
1,19,2016-11-30 00:00:00+00:00,598.053333
2,20,2016-12-31 00:00:00+00:00,686.743333
3,21,2017-01-31 00:00:00+00:00,738.986667
4,22,2017-02-28 00:00:00+00:00,887.996667
...,...,...,...
56,74,2021-06-30 00:00:00+00:00,25347.843333
57,75,2021-07-31 00:00:00+00:00,24324.596667
58,76,2021-08-31 00:00:00+00:00,32926.090000
59,77,2021-09-30 00:00:00+00:00,33857.720000


In [ ]:
inflation_BTC_monthly = pivoted_monthlyInflation.join(BTC_monthly['price_close']).reset_index().drop(['index'], axis=1)[0:60].drop(['level_0'], axis=1)
inflation_BTC_monthly.rename(columns = {'price_close':'price_close_BTC'}, inplace = True)
inflation_BTC_monthly
# Our pivoted inflation table is then joined to the price column of the Bitcoin dataframe, as the dates are now aligned.
# Columns that are not useful are also dropped from our dataframe using the 'drop()' function.
# We rename the 'price_close' column as this is also the name of the price column in our Ethereum dataframe, and thus the join function would not work otherwise.

,TIME,DEU,JPN,TUR,USA,price_close_BTC
0,2016-11-01,0.802407,0.5,7.000521,1.692537,532.686667
1,2016-12-01,1.504514,0.3,8.533056,2.074622,598.053333
2,2017-01-01,1.616162,0.4,9.218773,2.500042,686.743333
3,2017-02-01,1.913394,0.3,10.128290,2.737958,738.986667
4,2017-03-01,1.400000,0.2,11.291790,2.380612,887.996667
5,2017-04-01,1.698302,0.4,11.869620,2.199690,883.123333
6,2017-05-01,1.192843,0.4,11.722180,1.874878,983.233333
7,2017-06-01,1.390268,0.4,10.901080,1.633488,1514.836667
8,2017-07-01,1.384768,0.4,9.788379,1.727978,2067.993333
9,2017-08-01,1.584158,0.7,10.679300,1.938974,1900.401667


In [ ]:
url ='https://raw.githubusercontent.com/tm19784/tm19784.github.io/main/jsonFolder/rawPriceETH.json'
html = requests.get(url)
Ethereum_price_json = html.json()
ETH=pd.DataFrame(Ethereum_price_json)
ETH_useful=ETH[['time_close','price_close']]
ETH_useful
# This is exactly the same technique I used above, just for Ethereum in this case rather than Bitcoin.

,time_close,price_close
0,2016-10-20T20:18:26.6010000Z,10.18478
1,2016-10-30T23:46:16.0410000Z,9.68622
2,2016-11-09T21:38:05.5430000Z,9.05000
3,2016-11-19T20:45:29.0350000Z,7.98496
4,2016-11-29T22:08:00.0360000Z,6.89169
...,...,...
184,2021-11-13T23:59:52.6350460Z,3470.13000
185,2021-11-23T23:59:55.6560050Z,3246.49000
186,2021-12-03T23:59:58.9420140Z,3193.87000
187,2021-12-13T23:59:58.8397000Z,2866.93000


In [ ]:
ETH_useful['time_close'] = pd.to_datetime(ETH_useful['time_close'])
ETH_useful
ETH_monthly = ETH_useful.set_index('time_close').resample('1M').mean().reset_index()[0:60].drop(['time_close'], axis = 1)
ETH_monthly
# Again this is exactly the same technique as I used to resample Bitcoin's price data, just for Ethereum's instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,price_close
0,9.935500
1,7.975550
2,6.709893
3,8.442843
4,10.676927
5,29.256237
6,43.298287
7,91.308423
8,234.967380
9,163.496253


In [ ]:
inflation_BTC_ETH_monthly = inflation_BTC_monthly.join(ETH_monthly)
inflation_BTC_ETH_monthly
# Thus this join works as 'price_close_BTC' differs from 'price_close'.

,TIME,DEU,JPN,TUR,USA,price_close_BTC,price_close
0,2016-11-01,0.802407,0.5,7.000521,1.692537,532.686667,9.935500
1,2016-12-01,1.504514,0.3,8.533056,2.074622,598.053333,7.975550
2,2017-01-01,1.616162,0.4,9.218773,2.500042,686.743333,6.709893
3,2017-02-01,1.913394,0.3,10.128290,2.737958,738.986667,8.442843
4,2017-03-01,1.400000,0.2,11.291790,2.380612,887.996667,10.676927
5,2017-04-01,1.698302,0.4,11.869620,2.199690,883.123333,29.256237
6,2017-05-01,1.192843,0.4,11.722180,1.874878,983.233333,43.298287
7,2017-06-01,1.390268,0.4,10.901080,1.633488,1514.836667,91.308423
8,2017-07-01,1.384768,0.4,9.788379,1.727978,2067.993333,234.967380
9,2017-08-01,1.584158,0.7,10.679300,1.938974,1900.401667,163.496253


In [ ]:
for x in inflation_BTC_ETH_monthly.columns[1:7]:
  inflation_BTC_ETH_monthly[x+'_z_norm']= (inflation_BTC_ETH_monthly[x] - inflation_BTC_ETH_monthly[x].mean())/inflation_BTC_ETH_monthly[x].std()
inflation_BTC_ETH_monthly.rename(columns = {'DEU_z_norm': 'Germany_inflation_Z'}, inplace = True)
inflation_BTC_ETH_monthly.rename(columns = {'JPN_z_norm': 'Japan_inflation_Z'}, inplace = True)
inflation_BTC_ETH_monthly.rename(columns = {'TUR_z_norm': 'Turkey_inflation_Z'}, inplace = True)
inflation_BTC_ETH_monthly.rename(columns = {'USA_z_norm': 'USA_inflation_Z'}, inplace = True)
inflation_BTC_ETH_monthly.rename(columns = {'price_close_BTC_z_norm': 'Bitcoin_price_Z'}, inplace = True)
inflation_BTC_ETH_monthly.rename(columns = {'price_close_z_norm': 'Ethereum_price_Z'}, inplace = True)
inflation_BTC_ETH_monthly
# This is a 'for loop' for columns 1 through 7, i.e. all columns bar that showing the date.
# The loop creates a new column with the suffix '_z_norm' for each of these six variables.
# Normalising is done by taking away the mean of said variable from each variable's observation and dividing this by the variable's standard deviation.
# I then rename each normalised column as something more clear to what it is, to make the legend clearer on my GitHub without further editing.

,TIME,DEU,JPN,TUR,USA,price_close_BTC,price_close,Germany_inflation_Z,Japan_inflation_Z,Turkey_inflation_Z,USA_inflation_Z,Bitcoin_price_Z,Ethereum_price_Z
0,2016-11-01,0.802407,0.5,7.000521,1.692537,532.686667,9.935500,-0.781041,0.294647,-1.716531,-0.487159,-0.864629,-0.714800
1,2016-12-01,1.504514,0.3,8.533056,2.074622,598.053333,7.975550,-0.028818,-0.042092,-1.351271,-0.178274,-0.858306,-0.718145
2,2017-01-01,1.616162,0.4,9.218773,2.500042,686.743333,6.709893,0.090800,0.126277,-1.187840,0.165645,-0.849728,-0.720305
3,2017-02-01,1.913394,0.3,10.128290,2.737958,738.986667,8.442843,0.409249,-0.042092,-0.971068,0.357981,-0.844675,-0.717347
4,2017-03-01,1.400000,0.2,11.291790,2.380612,887.996667,10.676927,-0.140792,-0.210462,-0.693763,0.069095,-0.830262,-0.713535
5,2017-04-01,1.698302,0.4,11.869620,2.199690,883.123333,29.256237,0.178803,0.126277,-0.556045,-0.077166,-0.830733,-0.681826
6,2017-05-01,1.192843,0.4,11.722180,1.874878,983.233333,43.298287,-0.362736,0.126277,-0.591185,-0.339751,-0.821050,-0.657862
7,2017-06-01,1.390268,0.4,10.901080,1.633488,1514.836667,91.308423,-0.151219,0.126277,-0.786884,-0.534896,-0.769632,-0.575926
8,2017-07-01,1.384768,0.4,9.788379,1.727978,2067.993333,234.967380,-0.157111,0.126277,-1.052081,-0.458508,-0.716129,-0.330753
9,2017-08-01,1.584158,0.7,10.679300,1.938974,1900.401667,163.496253,0.056512,0.631386,-0.839742,-0.287934,-0.732339,-0.452728


In [ ]:
inflation_BTC_ETH_monthly_stack=inflation_BTC_ETH_monthly.set_index('TIME').stack().reset_index()
inflation_BTC_ETH_monthly_stack.columns=['TIME','Variable','Score']
inflation_BTC_ETH_monthly_stack['TIME']=pd.to_datetime(inflation_BTC_ETH_monthly_stack['TIME'])
inflation_BTC_ETH_monthly_stack=inflation_BTC_ETH_monthly_stack[inflation_BTC_ETH_monthly_stack['Variable'].str.contains('_Z')]
inflation_BTC_ETH_monthly_stack
# I then stack each of the normalised columns into one column; essentially the reverse of a pivot table.
# I then reset the date index after having to set it for stacking the columns, and then name each column their relevant names.
# I then only include observations in the 'Variable' column that contain in their string tags relating to variables being normalised.

,TIME,Variable,Score
6,2016-11-01,Germany_inflation_Z,-0.781041
7,2016-11-01,Japan_inflation_Z,0.294647
8,2016-11-01,Turkey_inflation_Z,-1.716531
9,2016-11-01,USA_inflation_Z,-0.487159
10,2016-11-01,Bitcoin_price_Z,-0.864629
...,...,...,...
715,2021-10-01,Japan_inflation_Z,-0.378832
716,2021-10-01,Turkey_inflation_Z,1.355512
717,2021-10-01,USA_inflation_Z,3.174450
718,2021-10-01,Bitcoin_price_Z,2.358688


In [ ]:
inflation_BTC_ETH_monthly_stack.to_csv('inflationCoins_normalisation.csv')
# I then export this stacked dataframe as a CSV, in order to download and then upload to my GitHub.